# Lec07 - spatial autocorrelation and regression

*   Section 1. Global and Local Moran's I
*   Section 2. Baseline linear regression
*   Section 3. Spatial regression with lag term (x)
*   Section 4. Spatial regression with lag term (y)



In [ ]:
# install the two packages
!pip install pysal 
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/4

In [2]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import statsmodels.api as sm

import pysal.explore as ps
from pysal.model import spreg
from pysal.lib import weights
from pysal.lib import cg as geometry
# import pysal as ps

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/autumnstar/opt/anaconda3/envs/Python_Tutorials/lib/python3.11/site-packages/spaghetti/network.py:40: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


In [3]:
# read data
df = pd.read_csv('SampleDataset/Florida_ct.csv', index_col = 0)
florida_shapefile = gpd.read_file('SampleDataset/tl_2020_12_tract/tl_2020_12_tract.shp') # read the shapefile

In [4]:
# view the data 
df.head()

,pop_total,sex_total,sex_male,sex_female,age_median,households,race_total,race_white,race_black,race_native,...,travel_walk_ratio,travel_work_home_ratio,edu_bachelor_ratio,edu_master_ratio,edu_phd_ratio,edu_higher_edu_ratio,employment_unemployed_ratio,vehicle_per_capita,vehicle_per_household,vacancy_ratio
0,2812.0,2812.0,1383.0,1429.0,39.4,931.0,2812.0,2086.0,517.0,0.0,...,0.014815,0.024242,0.183838,0.029798,0.003030,0.216667,0.286635,0.528094,1.595059,0.155938
1,4709.0,4709.0,2272.0,2437.0,34.2,1668.0,4709.0,2382.0,1953.0,0.0,...,0.022150,0.004615,0.135222,0.040245,0.003220,0.178686,0.318327,0.460183,1.299161,0.152869
2,5005.0,5005.0,2444.0,2561.0,34.1,1379.0,5005.0,2334.0,2206.0,224.0,...,0.026141,0.027913,0.213247,0.064620,0.007431,0.285299,0.366755,0.450949,1.636693,0.162211
3,6754.0,6754.0,2934.0,3820.0,31.3,2238.0,6754.0,4052.0,1671.0,326.0,...,0.052697,0.004054,0.093379,0.082510,0.012599,0.188488,0.314452,0.474830,1.432976,0.178716
4,3021.0,3021.0,1695.0,1326.0,44.1,1364.0,3021.0,2861.0,121.0,0.0,...,0.003014,0.013059,0.219868,0.138631,0.007064,0.365563,0.218447,0.659053,1.459677,0.335930


In [5]:
# view the shapefile
florida_shapefile.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,12,011,110403,12011110403,1104.03,Census Tract 1104.03,G5020,S,1323099,0,+25.9882712,-080.2428385,"POLYGON ((-80.24758 25.99480, -80.24754 25.994..."
1,12,011,060114,12011060114,601.14,Census Tract 601.14,G5020,S,2598912,0,+26.1864137,-080.2598783,"POLYGON ((-80.26810 26.19368, -80.26702 26.193..."
2,12,011,060120,12011060120,601.20,Census Tract 601.20,G5020,S,12814719,1823779,+26.1433192,-080.3341825,"POLYGON ((-80.36670 26.12828, -80.36649 26.128..."
3,12,011,110347,12011110347,1103.47,Census Tract 1103.47,G5020,S,2846117,545293,+26.0230637,-080.4006685,"POLYGON ((-80.40957 26.03541, -80.40878 26.035..."
4,12,011,020421,12011020421,204.21,Census Tract 204.21,G5020,S,1060862,16632,+26.2129463,-080.2358809,"POLYGON ((-80.24061 26.22083, -80.24056 26.220..."


In [6]:
# processing the data
# adjust the object types to facilitate the merge
florida_shapefile['GEOID'] = florida_shapefile.GEOID.astype('int64')

# combine the dataframe with the shapefile.
# Note that it is important to choose how - e.g., inner, right, left, etc. Here I choose 'left' for teaching purposes.
df_shp = florida_shapefile.merge(df,
                                 how = 'left',
                                 left_on = 'GEOID',
                                 right_on = 'full_ct_fips') 

# With the current approach, I will fill in ZEROS into the NaN values.
# However, it is NOT necessarily the best approach.
df_shp = df_shp.fillna(0.0)

df_shp

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,travel_walk_ratio,travel_work_home_ratio,edu_bachelor_ratio,edu_master_ratio,edu_phd_ratio,edu_higher_edu_ratio,employment_unemployed_ratio,vehicle_per_capita,vehicle_per_household,vacancy_ratio
0,12,011,110403,12011110403,1104.03,Census Tract 1104.03,G5020,S,1323099,0,...,0.000000,0.017004,0.091483,0.060147,0.013899,0.165529,0.278519,0.553838,1.937228,0.030723
1,12,011,060114,12011060114,601.14,Census Tract 601.14,G5020,S,2598912,0,...,0.010400,0.050442,0.253958,0.054749,0.023307,0.332014,0.233480,0.607103,1.957252,0.108844
2,12,011,060120,12011060120,601.20,Census Tract 601.20,G5020,S,12814719,1823779,...,0.015806,0.093356,0.323871,0.118865,0.013207,0.455943,0.258939,0.554582,1.674524,0.215953
3,12,011,110347,12011110347,1103.47,Census Tract 1103.47,G5020,S,2846117,545293,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,12,011,020421,12011020421,204.21,Census Tract 204.21,G5020,S,1060862,16632,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5155,12,015,020700,12015020700,207,Census Tract 207,G5020,S,2619776,106556,...,0.016979,0.063450,0.048280,0.014137,0.006402,0.068818,0.702392,0.240077,0.486310,0.170811
5156,12,005,001900,12005001900,19,Census Tract 19,G5020,S,3493618,2801264,...,0.000000,0.042712,0.272564,0.104603,0.002690,0.379857,0.352463,0.498792,1.173037,0.060650
5157,12,005,000600,12005000600,6,Census Tract 6,G5020,S,54190724,9090097,...,0.039185,0.095611,0.194805,0.131494,0.018669,0.344968,0.489378,0.460317,0.938235,0.643045
5158,12,005,000500,12005000500,5,Census Tract 5,G5020,S,337874718,28527612,...,0.000000,0.024557,0.108001,0.060415,0.013996,0.182412,0.331173,0.485219,1.382467,0.201167


# Secion 1. Spatial autocorrelation - Moran's I 



In [7]:
# Create the weighting matrix
w = weights.Queen.from_dataframe(df_shp)
w.transform = 'R' # row normalize the matrix
w

/var/folders/jh/p1jvq4k96b78klmgl3dq5jwc0000gn/T/ipykernel_41399/3209979706.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = weights.Queen.from_dataframe(df_shp)


('WARNING: ', 1636, ' is an island (no neighbors)')


/Users/autumnstar/opt/anaconda3/envs/Python_Tutorials/lib/python3.11/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 1636.
  warnings.warn(message)


In [8]:
# compute Moran's I
# The value indicates a relatively strong spatial autocorrelation among the property values.
I_stats = ps.esda.Moran(df_shp['property_value_median'], w)
print(I_stats.I)

0.38291116263972147


In [9]:
# compute local Moran's I
I_local_stats = ps.esda.Moran_Local(df_shp['property_value_median'], w)
I_local_stats.Is[:10] # show the first 10 local Moran's I

array([ 0.04817307,  0.13429403,  0.60396743, -0.55434325,  0.04132121,
       -0.62603027, -0.1291442 , -0.21423089,  0.14755619,  0.31011156])

### **Exercise 1.** Create an adjacency matrix using the Rook's rule. Then compute global and local Moran's I for median household income.

# Section 2. Baseline Linear Regression

In [10]:
# choose the independent variable
X = df_shp['inc_median_household']
# add a constant to the independent variable
X = sm.add_constant(X)

# choose the dependent var
y = df_shp['property_value_median']

In [11]:
# fitting the model.
# initialize the model using the independent and dependent variables
model = sm.OLS(y, X)
# fitting the model
results = model.fit()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
# summary of the regression.
# R square is different because of the data processing. 
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     property_value_median   R-squared:                       0.726
Model:                               OLS   Adj. R-squared:                  0.726
Method:                    Least Squares   F-statistic:                 1.370e+04
Date:                   Tue, 28 Nov 2023   Prob (F-statistic):               0.00
Time:                           16:53:31   Log-Likelihood:                -66585.
No. Observations:                   5160   AIC:                         1.332e+05
Df Residuals:                       5158   BIC:                         1.332e+05
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
co

In [13]:
# The result above can be replicated by using Pysal
m1 = spreg.OLS(df_shp[['property_value_median']].values, df_shp[['inc_median_household']].values, 
               name_y = 'property_value_median', name_x = ['inc_median_household'])


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [14]:
# Pysal summary 
print(m1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :property_value_median                Number of Observations:        5160
Mean dependent var  : 149335.0940                Number of Variables   :           2
S.D. dependent var  : 185957.9336                Degrees of Freedom    :        5158
R-squared           :      0.7264
Adjusted R-squared  :      0.7264
Sum squared residual: 4.88016e+13                F-statistic           :  13697.6754
Sigma-square        :9461344528.059                Prob(F-statistic)     :           0
S.E. of regression  :   97269.443                Log likelihood        :  -66584.562
Sigma-square ML     :9457677340.258                Akaike info criterion :  133173.124
S.E of regression ML:  97250.5904                Schwarz criterion     :  133186.221

-----------------------------------------------

# Section 3. Spatial lag model using x. 

In [15]:
# KNN rule to create the adjacency matrix.
# Then we normalize each row. 
w = weights.KNN.from_dataframe(df_shp, k=4)
w.transform = 'R'
w

In [17]:
# create the spatial lag term
# it is in fact the spatial averaging over the neighboring census tracts.
# Since the W matrix is row normalized, it provides the correct magnitude.
inc_median_household_spatial_lag = weights.spatial_lag.lag_spatial(w, df_shp['inc_median_household'].values)

# add the spatial lag term to the dataframe
df_shp['inc_median_household_spatial_lag'] = inc_median_household_spatial_lag

In [18]:
# rerun the regression by incorporating the spatial lag term.
model = spreg.OLS(df_shp[['property_value_median']].values, df_shp[['inc_median_household', 'inc_median_household_spatial_lag']].values, 
                  name_y = 'property_value_median', name_x = ['inc_median_household', 'inc_median_household_spatial_lag'])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [19]:
# print the OLS result.
# note that the performance increases slightly, but not much. 
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :property_value_median                Number of Observations:        5160
Mean dependent var  : 149335.0940                Number of Variables   :           3
S.D. dependent var  : 185957.9336                Degrees of Freedom    :        5157
R-squared           :      0.7276
Adjusted R-squared  :      0.7275
Sum squared residual:  4.8596e+13                F-statistic           :   6887.3841
Sigma-square        :9423316421.757                Prob(F-statistic)     :           0
S.E. of regression  :   97073.768                Log likelihood        :  -66573.671
Sigma-square ML     :9417837749.419                Akaike info criterion :  133153.342
S.E of regression ML:  97045.5447                Schwarz criterion     :  133172.988

-----------------------------------------------

We shall see that the spatial lag model using exogeneous variable is roughly the same as linear regressions. This is why researchers do not discuss this model that much in the spatial econometrics class. However, it is an important step for us to grasp the gist (e.g. local message passing) in spatial regressions. 

### **Exercise 2.** Instead of using only the inc_median_household, please try to add other control variables to predict property values. For example, please include median household income and spatial lag of median household income, along with household size, travel behavior, and total population for the regression. 

# Section 4. Spatial lag model using y. 

In [20]:
# KNN rule to create the adjacency matrix.
# Then we normalize each row. 
w = weights.KNN.from_dataframe(df_shp, k=4)
w.transform = 'R'
w

In [21]:
# specify the model
# note that the syntax is different from the previous case. It is caused by the difference in their estimation methods. 
model = spreg.GM_Lag(df_shp[['property_value_median']].values, df_shp[['inc_median_household']].values,
                     w = w, 
                     name_y = 'property_value_median', name_x = ['inc_median_household'])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [22]:
# print the result.
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :property_value_median                Number of Observations:        5160
Mean dependent var  : 149335.0940                Number of Variables   :           3
S.D. dependent var  : 185957.9336                Degrees of Freedom    :        5157
Pseudo R-squared    :      0.7426
Spatial Pseudo R-squared:  0.7276

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT    -22693.31038      2250.33615       -10.08441         0.00000
inc_median_household         4.31433         0.04464        96.64288         0.00000
W_property_value_median         0

### **Exercise 3.** Instead of using only the inc_median_household, please try to add other control variables to predict property values. For example, please include median household income, along with household size, travel behavior, and total population for this spatial lag regression using the endogeneous variable y. 